In [ ]:
%matplotlib inline
import matplotlib as plt
from ipywidgets import widgets

### Normal distribution

The normal distribution is a symmetric unimodal Bell-shaped distribution which is described by two parameters: the mean and the standard distribution. The mean indicates the shift on the x axis, whereas the standard distribution determines the width of the central part of the distribution. 

In [ ]:
number_dist = widgets.Dropdown(
    options=['1', '2', '3'],
    value='1',
    description='Number of distributions:',
    disabled=False,
)

def define_input(description, value):
    return widgets.BoundedFloatText(
        value=value,
        description=description,
        disabled=False
    )

widgets.VBox([
    number_dist,
    widgets.HBox([define_input('Mean 1: ', 0), define_input('Standard deviation 1: ', 1)]),
    widgets.HBox([define_input('Mean 2: ', -1), define_input('Standard deviation 2: ', 1)]),
    widgets.HBox([define_input('Mean 3: ', 1), define_input('Standard deviation 3: ', 1)])
])


### Bernoulli distribution

### Geometric distribution

### Binomial distribution

### Poisson distribution

### Negative binomial distribution